# VGG

In [15]:
import os
import threading
import queue
import tkinter as tk
from tkinter import scrolledtext, messagebox, ttk
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from scapy.all import sniff, IP, get_if_list
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Dense, Flatten, Input, concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import time
from scapy.utils import wrpcap

# ─── Configurații directoare ────────────────────────────────────────────────────
MODEL_DIR = "./saved_model"
MODEL_FILE_PATH = os.path.join(MODEL_DIR, "model_traffic_final_model.h5")
LOG_DIR = "./analysis_logs"

# ─── Definire coloane și mapping atacuri ────────────────────────────────────────
columns = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent',
    'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
    'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
    'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
    'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate'
]

def group_attack_label(attack):
    attack = attack.lower()
    if attack in ['satan', 'portsweep', 'nmap', 'ipsweep']:
        return 'Probe'
    if attack in ['spy', 'phf', 'multihop', 'imap', 'guess_passwd', 'ftp_write', 'warezmaster', 'warezclient']:
        return 'R2L'
    if attack in ['rootkit', 'perl', 'loadmodule', 'buffer_overflow']:
        return 'U2R'
    if attack in ['teardrop', 'smurf', 'pod', 'neptune', 'land', 'back']:
        return 'DoS'
    if attack == 'normal':
        return 'NORMAL'
    return 'NORMAL'

# ─── Încărcare și preprocesare pentru antrenament ───────────────────────────────
def load_and_preprocess_train_data(path):
    df = pd.read_csv(path, header=None)
    df.columns = columns + ['attack', 'level']
    X = df.iloc[:, :-2]  # Exclude 'attack' and 'level'
    y_raw = df['attack'].map(group_attack_label)

    # label-encode categorice
    encoders = {}
    for col in X.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])
        encoders[col] = le

    le_y = LabelEncoder()
    y = le_y.fit_transform(y_raw)

    scaler = MinMaxScaler()
    Xs = scaler.fit_transform(X)
    Xs = Xs.reshape(Xs.shape[0], Xs.shape[1], 1)
    return Xs, y, encoders, le_y, scaler, df

# ─── Definirea unui bloc Inception pentru date 1D ─────────────────────────────────
def inception_block(x, filters):
    branch1 = Conv1D(filters, kernel_size=1, padding='same', activation='relu')(x)
    branch3 = Conv1D(filters, kernel_size=1, padding='same', activation='relu')(x)
    branch3 = Conv1D(filters, kernel_size=3, padding='same', activation='relu')(branch3)
    branch5 = Conv1D(filters, kernel_size=1, padding='same', activation='relu')(x)
    branch5 = Conv1D(filters, kernel_size=5, padding='same', activation='relu')(branch5)
    branch_pool = MaxPooling1D(pool_size=3, strides=1, padding='same')(x)
    branch_pool = Conv1D(filters, kernel_size=1, padding='same', activation='relu')(branch_pool)
    x = concatenate([branch1, branch3, branch5, branch_pool], axis=-1)
    return x

# ─── Crearea modelului VGG16IncepNet pentru date 1D ──────────────────────────────
def create_vgg16incepnet_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = inception_block(x, filters=64)
    x = Conv1D(256, kernel_size=3, activation='relu', padding='same')(x)
    x = Conv1D(256, kernel_size=3, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(1e-4), metrics=["accuracy"])
    return model

# ─── Încărcare date și model ──────────────────────────────────────────────────
train_path = "../Data/nsl-kdd/KDDTrain+.txt"
test_path = "../Data/nsl-kdd/KDDTest+.txt"

X_train, y_train, label_encoders, label_encoder_y, scaler, df_train = load_and_preprocess_train_data(train_path)
X_test, y_test, _, _, _, df_test = load_and_preprocess_train_data(test_path)

num_classes = len(np.unique(y_train))
print(f"Numărul de clase: {num_classes}")
print(f"Dimensiunea setului de antrenament: {X_train.shape}")

# Împărțirea datelor de antrenament în set de antrenament și validare
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

if os.path.exists(MODEL_FILE_PATH):
    model = load_model(MODEL_FILE_PATH, compile=False)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(1e-4), metrics=["accuracy"])
else:
    os.makedirs(MODEL_DIR, exist_ok=True)
    model = create_vgg16incepnet_model(X_train.shape[1:], num_classes)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping])
    model.save(MODEL_FILE_PATH)

# Evaluarea modelului pe setul de testare
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss pe setul de testare: {loss}")
print(f"Acuratețe pe setul de testare: {accuracy}")

# ─── Încărcare și preprocesare date de testare ─────────────────────────────────
X_test, y_test, _, _, _, df_test = load_and_preprocess_train_data(test_path)

# Adaugă coloanele 'src_ip' și 'dst_ip' cu valori fictive
df_test['src_ip'] = '192.168.1.1'
df_test['dst_ip'] = '192.168.1.2'

# ─── Funcții de preprocesare live ──────────────────────────────────────────────
def extract_features(packet):
    feat = {c: 0 for c in columns}
    if IP in packet:
        feat['protocol_type'] = packet[IP].proto
        feat['src_bytes'] = len(packet[IP].payload)
        feat['dst_bytes'] = len(packet[IP].payload)
    return pd.DataFrame([feat], columns=columns)

def extract_additional_features(packet):
    feat = {}
    if IP in packet:
        feat['src_ip'] = packet[IP].src
        feat['dst_ip'] = packet[IP].dst
        feat['src_port'] = packet[IP].sport
        feat['dst_port'] = packet[IP].dport
    return feat

def preprocess_and_predict(data, threshold=94):
    if isinstance(data, pd.DataFrame):
        arr = data.values  # din DataFrame în numpy
    else:
        arr = extract_features(data).values  # pentru pachete live
    arr_s = scaler.transform(arr)  # normalizare
    arr_s = arr_s.reshape(1, arr_s.shape[1], 1)  # reshape corect
    probs = model.predict(arr_s, verbose=0)[0]
    idx = np.argmax(probs)
    conf = probs[idx] * 100
    if conf < threshold:
        return "Trafic Legitim", conf
    cat = label_encoder_y.inverse_transform([idx])[0]
    return f"Atac: {cat}", conf

# ─── Clasa principală Tkinter cu coadă de mesaje ─────────────────────────────
class LiveTrafficAnalyzer:
    def __init__(self, root):
        self.root = root
        root.title("Live Traffic Analyzer")
        # widget-uri
        ttk.Label(root, text="Selectează sursa datelor:").pack(pady=5)
        self.source_var = tk.StringVar(value="live")
        self.source_cb = ttk.Combobox(root, textvariable=self.source_var, values=["live", "random"], state="readonly")
        self.source_cb.pack(pady=5)

        ttk.Label(root, text="Selectează interfața:").pack(pady=5)
        self.interface_var = tk.StringVar()
        self.iface_cb = ttk.Combobox(root, textvariable=self.interface_var, state="readonly")
        self.iface_cb.pack(pady=5)

        ttk.Label(root, text="Selectează pragul minim de afișare (%):").pack(pady=5)
        self.threshold_var = tk.DoubleVar(value=94.0)
        self.threshold_entry = ttk.Entry(root, textvariable=self.threshold_var)
        self.threshold_entry.pack(pady=5)

        self.start_btn = tk.Button(root, text="Start", command=self.start_analysis)
        self.start_btn.pack(pady=5)
        self.stop_btn = tk.Button(root, text="Stop", command=self.stop_analysis, state="disabled")
        self.stop_btn.pack(pady=5)

        # Treeview pentru afișarea pachetelor
        self.tree = ttk.Treeview(root, columns=("No", "Time", "Source", "Destination", "Protocol", "Length", "Info", "Prediction"), show="headings")
        self.tree.heading("No", text="No")
        self.tree.heading("Time", text="Time")
        self.tree.heading("Source", text="Source")
        self.tree.heading("Destination", text="Destination")
        self.tree.heading("Protocol", text="Protocol")
        self.tree.heading("Length", text="Length")
        self.tree.heading("Info", text="Info")
        self.tree.heading("Prediction", text="Prediction")
        self.tree.pack(pady=5, fill=tk.BOTH, expand=True)

        # Ajustează lățimea coloanelor
        self.tree.column("Protocol", width=int(self.tree.column("Protocol", width=None) / 3))
        self.tree.column("Length", width=int(self.tree.column("Length", width=None) / 3))

        self.packet_view = scrolledtext.ScrolledText(root, width=80, height=10)
        self.packet_view.pack(pady=5)
        self.packet_view.insert(tk.END, "Detalii pachet:\n"+"-"*60+"\n")

        # setări interne
        self.queue = queue.Queue()
        self.running = False
        self.stop_event = threading.Event()
        self.log_file = None
        self.sniff_thread = None
        self.random_thread = None
        self.packet_counter = 0
        self.start_time = None

        # populație interfețe
        ifaces = get_if_list()
        if not ifaces:
            messagebox.showerror("Eroare", "Nicio interfață disponibilă (rulează ca admin).")
            self.start_btn.config(state="disabled")
        else:
            self.iface_cb['values'] = ifaces
            self.iface_cb.current(0)

        # job periodic
        self.root.after(100, self.process_queue)

    def process_queue(self):
        while not self.queue.empty():
            item = self.queue.get()
            if len(item) == 4:
                ts, atype, conf, summary = item
                details = ""
            else:
                ts, atype, conf, summary, details = item
                details_parts = details.split('\t')
                if len(details_parts) >= 6:
                    prediction_info = f"{atype} ({conf:.2f}%)" if conf >= self.threshold_var.get() else ""
                    self.tree.insert("", "end", values=(self.packet_counter, ts, details_parts[1], details_parts[2], details_parts[3], details_parts[4], summary, prediction_info))
                    self.packet_counter += 1
            self.packet_view.delete("1.0", tk.END)
            self.packet_view.insert(tk.END, f"{summary}\n{details}\n" + "-"*40 + "\n")
        self.root.after(100, self.process_queue)

    def start_analysis(self):
        source = self.source_var.get()
        iface = self.interface_var.get()
        if source == "live" and not iface:
            messagebox.showerror("Eroare", "Selectează o interfață.")
            return
        self.running = True
        self.stop_event.clear()
        self.start_btn.config(state="disabled")
        self.stop_btn.config(state="normal")
        os.makedirs(LOG_DIR, exist_ok=True)
        fname = datetime.now().strftime("log_%Y%m%d_%H%M%S")
        self.log_file = os.path.join(LOG_DIR, f"{fname}.txt")
        self.pcap_file = os.path.join(LOG_DIR, f"{fname}.pcapng")

        # Initializează timpul de start
        self.start_time = datetime.now()

        if source == "live":
            # thread de sniff
            self.sniff_thread = threading.Thread(target=self.sniff_packets, args=(iface,), daemon=True)
            self.sniff_thread.start()
        elif source == "random":
            # thread pentru mostre random
            self.random_thread = threading.Thread(target=self.process_random_samples, daemon=True)
            self.random_thread.start()

    def stop_analysis(self):
        self.stop_event.set()
        self.running = False
        self.start_btn.config(state="normal")
        self.stop_btn.config(state="disabled")
        if self.sniff_thread is not None:
            self.sniff_thread.join()  # Așteaptă ca thread-ul de sniff să se termine

    def sniff_packets(self, iface):
        while not self.stop_event.is_set():
            sniff(filter="ip", prn=self.handle_packet, store=0, iface=iface, timeout=1)

    def process_random_samples(self):
        while not self.stop_event.is_set():
            index = np.random.randint(0, len(X_test))
            sample_data = X_test[index]
            elapsed_time = datetime.now() - self.start_time
            ts = f"{elapsed_time.seconds + elapsed_time.microseconds / 1e6:.4f}"
            try:
                atype, conf = preprocess_and_predict(sample_data, threshold=self.threshold_var.get())
            except Exception as e:
                atype, conf = f"Eroare: {e}", 0.0

            # Extrage detaliile despre pachet
            src_ip = df_test.iloc[index]['src_ip']
            dst_ip = df_test.iloc[index]['dst_ip']
            protocol_type = df_test.iloc[index]['protocol_type']
            service = df_test.iloc[index]['service']
            flag = df_test.iloc[index]['flag']
            src_bytes = df_test.iloc[index]['src_bytes']
            dst_bytes = df_test.iloc[index]['dst_bytes']

            details = (f"{self.packet_counter}\t{src_ip}\t{dst_ip}\t{protocol_type}\t{service}\t{flag}\t"
                       f"{src_bytes}\t{dst_bytes}")
            summary = f"protocol_type: {protocol_type} | service: {service} | flag: {flag} | src_bytes: {src_bytes} | dst_bytes: {dst_bytes}"

            # pune în coadă pentru UI
            self.queue.put((ts, atype, conf, summary, details))
            # log la fișier
            with open(self.log_file, "a", encoding="utf-8") as f:
                f.write(f"{ts}\t{atype}\t{conf:.2f}%\n{details}\n{'-'*40}\n")
            time.sleep(3)  # Așteaptă 3 secunde înainte de a procesa următoarea mostră

    def handle_packet(self, pkt):
        if IP in pkt:
            elapsed_time = datetime.now() - self.start_time
            ts = f"{elapsed_time.seconds + elapsed_time.microseconds / 1e6:.4f}"
            try:
                atype, conf = preprocess_and_predict(pkt, threshold=self.threshold_var.get())
            except Exception as e:
                atype, conf = f"Eroare: {e}", 0.0

            additional_features = extract_additional_features(pkt)
            details = (f"{self.packet_counter}\t{additional_features['src_ip']}:{additional_features['src_port']}\t"
                       f"{additional_features['dst_ip']}:{additional_features['dst_port']}\t"
                       f"{pkt[IP].proto}\t{len(pkt)}\t{pkt.summary()}")
            # pune în coadă pentru UI
            self.queue.put((ts, atype, conf, pkt.summary(), details))
            # log la fișier
            with open(self.log_file, "a", encoding="utf-8") as f:
                f.write(f"{ts}\t{atype}\t{conf:.2f}%\n{details}\n{'-'*40}\n")
            # scrie pachetul în fișierul .pcapng
            wrpcap(self.pcap_file, [pkt], append=True)

if __name__ == "__main__":
    root = tk.Tk()
    app = LiveTrafficAnalyzer(root)
    root.mainloop()


Numărul de clase: 5
Dimensiunea setului de antrenament: (125973, 41, 1)


ValueError: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, 42, 1), found shape=(None, 41)

# Final

In [7]:
import os
import threading
import queue
import tkinter as tk
from tkinter import scrolledtext, messagebox, ttk
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from scapy.all import sniff, IP, get_if_list
from tensorflow.keras.models import load_model, Sequential, Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Dense, LSTM, Input, Attention, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import time
from scapy.utils import wrpcap

# ─── Configurații directoare ────────────────────────────────────────────────────
MODEL_DIR = "./saved_model"
MODEL_FILE_PATH = os.path.join(MODEL_DIR, "model_traffic_final_model.h5")
LOG_DIR = "./analysis_logs"

# ─── Definire coloane și mapping atacuri ────────────────────────────────────────
columns = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent',
    'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
    'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
    'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
    'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate'
]

def group_attack_label(attack):
    attack = attack.lower()
    if attack in ['satan', 'portsweep', 'nmap', 'jpsweep']:
        return 'Probe'
    if attack in ['spy', 'phf', 'multihop', 'imap', 'guess_passwd', 'ftp_write', 'warezmaster', 'warezclient']:
        return 'R2L'
    if attack in ['rootkit', 'perl', 'loadmodule', 'buffer_overflow']:
        return 'U2R'
    if attack in ['teardrop', 'smurf', 'pod', 'neptune', 'land', 'back']:
        return 'DoS'
    if attack == 'normal':
        return 'NORMAL'
    return 'NORMAL'

# ─── Încărcare și preprocesare pentru antrenament ───────────────────────────────
def load_and_preprocess_train_data(path):
    df = pd.read_csv(path, header=None)
    df.columns = columns + ['attack', 'level']
    X = df.iloc[:, :-2]  # Exclude 'attack' and 'level'
    y_raw = df['attack'].map(group_attack_label)

    # label-encode categorice
    encoders = {}
    for col in X.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])
        encoders[col] = le

    le_y = LabelEncoder()
    y = le_y.fit_transform(y_raw)

    scaler = MinMaxScaler()
    Xs = scaler.fit_transform(X)
    Xs = Xs.reshape(Xs.shape[0], Xs.shape[1], 1)
    return Xs, y, encoders, le_y, scaler, df

# ─── Definire model NTAM-LSTM ───────────────────────────────────────────────────
def create_ntam_lstm_model(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    conv_layer = Conv1D(128, 3, padding="same", activation="relu")(input_layer)
    pooling_layer = MaxPooling1D(2)(conv_layer)
    lstm_layer = LSTM(64, return_sequences=True)(pooling_layer)
    attention_layer = Attention()([lstm_layer, lstm_layer])
    global_avg_pool = GlobalAveragePooling1D()(attention_layer)
    dropout_layer = Dropout(0.1)(global_avg_pool)
    output_layer = Dense(num_classes, activation="softmax")(dropout_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(1e-4), metrics=["accuracy"])
    return model

# ─── Încărcare date și model ──────────────────────────────────────────────────
train_path = "../Data/nsl-kdd/KDDTrain+.txt"
X_train, y_train, label_encoders, label_encoder_y, scaler, df_train = load_and_preprocess_train_data(train_path)
num_classes = len(np.unique(y_train))

if os.path.exists(MODEL_FILE_PATH):
    model = load_model(MODEL_FILE_PATH, compile=False)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(1e-4), metrics=["accuracy"])
else:
    os.makedirs(MODEL_DIR, exist_ok=True)
    model = create_ntam_lstm_model(X_train.shape[1:], num_classes)
    es = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[es])
    model.save(MODEL_FILE_PATH)

# ─── Încărcare și preprocesare date de testare ─────────────────────────────────
test_path = "../Data/nsl-kdd/KDDTest+.txt"
X_test, y_test, _, _, _, df_test = load_and_preprocess_train_data(test_path)

# Adaugă coloanele 'src_ip' și 'dst_ip' cu valori fictive
df_test['src_ip'] = '192.168.1.1'
df_test['dst_ip'] = '192.168.1.2'

# ─── Funcții de preprocesare live ──────────────────────────────────────────────
def extract_features(packet):
    feat = {c: 0 for c in columns}
    if IP in packet:
        feat['protocol_type'] = packet[IP].proto
        feat['src_bytes'] = len(packet[IP].payload)
        feat['dst_bytes'] = len(packet[IP].payload)
    return pd.DataFrame([feat], columns=columns)

def extract_additional_features(packet):
    feat = {}
    if IP in packet:
        feat['src_ip'] = packet[IP].src
        feat['dst_ip'] = packet[IP].dst
        feat['src_port'] = packet[IP].sport
        feat['dst_port'] = packet[IP].dport
    return feat

def preprocess_and_predict(data, threshold=94):
    if isinstance(data, pd.DataFrame):
        arr = data.values  # din DataFrame în numpy
    else:
        arr = extract_features(data).values  # pentru pachete live
    arr_s = scaler.transform(arr)  # normalizare
    arr_s = arr_s.reshape(1, arr_s.shape[1], 1)  # reshape corect
    probs = model.predict(arr_s, verbose=0)[0]
    idx = np.argmax(probs)
    conf = probs[idx] * 100
    if conf < threshold:
        return "Trafic Legitim", conf
    cat = label_encoder_y.inverse_transform([idx])[0]
    return f"Atac: {cat}", conf

# ─── Clasa principală Tkinter cu coadă de mesaje ─────────────────────────────
class LiveTrafficAnalyzer:
    def __init__(self, root):
        self.root = root
        root.title("Live Traffic Analyzer")
        # widget-uri
        ttk.Label(root, text="Selectează sursa datelor:").pack(pady=5)
        self.source_var = tk.StringVar(value="live")
        self.source_cb = ttk.Combobox(root, textvariable=self.source_var, values=["live", "random"], state="readonly")
        self.source_cb.pack(pady=5)

        ttk.Label(root, text="Selectează interfața:").pack(pady=5)
        self.interface_var = tk.StringVar()
        self.iface_cb = ttk.Combobox(root, textvariable=self.interface_var, state="readonly")
        self.iface_cb.pack(pady=5)

        ttk.Label(root, text="Selectează pragul minim de afișare (%):").pack(pady=5)
        self.threshold_var = tk.DoubleVar(value=94.0)
        self.threshold_entry = ttk.Entry(root, textvariable=self.threshold_var)
        self.threshold_entry.pack(pady=5)

        self.start_btn = tk.Button(root, text="Start", command=self.start_analysis)
        self.start_btn.pack(pady=5)
        self.stop_btn = tk.Button(root, text="Stop", command=self.stop_analysis, state="disabled")
        self.stop_btn.pack(pady=5)

        # Treeview pentru afișarea pachetelor
        self.tree = ttk.Treeview(root, columns=("No", "Time", "Source", "Destination", "Protocol", "Length", "Info", "Prediction"), show="headings")
        self.tree.heading("No", text="No")
        self.tree.heading("Time", text="Time")
        self.tree.heading("Source", text="Source")
        self.tree.heading("Destination", text="Destination")
        self.tree.heading("Protocol", text="Protocol")
        self.tree.heading("Length", text="Length")
        self.tree.heading("Info", text="Info")
        self.tree.heading("Prediction", text="Prediction")
        self.tree.pack(pady=5, fill=tk.BOTH, expand=True)

        # Ajustează lățimea coloanelor
        self.tree.column("Protocol", width=int(self.tree.column("Protocol", width=None) / 3))
        self.tree.column("Length", width=int(self.tree.column("Length", width=None) / 3))

        self.packet_view = scrolledtext.ScrolledText(root, width=80, height=10)
        self.packet_view.pack(pady=5)
        self.packet_view.insert(tk.END, "Detalii pachet:\n"+"-"*60+"\n")

        # setări interne
        self.queue = queue.Queue()
        self.running = False
        self.stop_event = threading.Event()
        self.log_file = None
        self.sniff_thread = None
        self.random_thread = None
        self.packet_counter = 0
        self.start_time = None

        # populație interfețe
        ifaces = get_if_list()
        if not ifaces:
            messagebox.showerror("Eroare", "Nicio interfață disponibilă (rulează ca admin).")
            self.start_btn.config(state="disabled")
        else:
            self.iface_cb['values'] = ifaces
            self.iface_cb.current(0)

        # job periodic
        self.root.after(100, self.process_queue)

    def process_queue(self):
        while not self.queue.empty():
            item = self.queue.get()
            if len(item) == 4:
                ts, atype, conf, summary = item
                details = ""
            else:
                ts, atype, conf, summary, details = item
                details_parts = details.split('\t')
                if len(details_parts) >= 6:
                    prediction_info = f"{atype} ({conf:.2f}%)" if conf >= self.threshold_var.get() else ""
                    self.tree.insert("", "end", values=(self.packet_counter, ts, details_parts[1], details_parts[2], details_parts[3], details_parts[4], summary, prediction_info))
                    self.packet_counter += 1
            self.packet_view.delete("1.0", tk.END)
            self.packet_view.insert(tk.END, f"{summary}\n{details}\n" + "-"*40 + "\n")
        self.root.after(100, self.process_queue)

    def start_analysis(self):
        source = self.source_var.get()
        iface = self.interface_var.get()
        if source == "live" and not iface:
            messagebox.showerror("Eroare", "Selectează o interfață.")
            return
        self.running = True
        self.stop_event.clear()
        self.start_btn.config(state="disabled")
        self.stop_btn.config(state="normal")
        os.makedirs(LOG_DIR, exist_ok=True)
        fname = datetime.now().strftime("log_%Y%m%d_%H%M%S")
        self.log_file = os.path.join(LOG_DIR, f"{fname}.txt")
        self.pcap_file = os.path.join(LOG_DIR, f"{fname}.pcapng")

        # Initializează timpul de start
        self.start_time = datetime.now()

        if source == "live":
            # thread de sniff
            self.sniff_thread = threading.Thread(target=self.sniff_packets, args=(iface,), daemon=True)
            self.sniff_thread.start()
        elif source == "random":
            # thread pentru mostre random
            self.random_thread = threading.Thread(target=self.process_random_samples, daemon=True)
            self.random_thread.start()

    def stop_analysis(self):
        self.stop_event.set()
        self.running = False
        self.start_btn.config(state="normal")
        self.stop_btn.config(state="disabled")
        if self.sniff_thread is not None:
            self.sniff_thread.join()  # Așteaptă ca thread-ul de sniff să se termine

    def sniff_packets(self, iface):
        while not self.stop_event.is_set():
            sniff(filter="ip", prn=self.handle_packet, store=0, iface=iface, timeout=1)

    def process_random_samples(self):
        while not self.stop_event.is_set():
            index = np.random.randint(0, len(X_test))
            sample_data = X_test[index]
            elapsed_time = datetime.now() - self.start_time
            ts = f"{elapsed_time.seconds + elapsed_time.microseconds / 1e6:.4f}"
            try:
                atype, conf = preprocess_and_predict(sample_data, threshold=self.threshold_var.get())
            except Exception as e:
                atype, conf = f"Eroare: {e}", 0.0

            # Extrage detaliile despre pachet
            src_ip = df_test.iloc[index]['src_ip']
            dst_ip = df_test.iloc[index]['dst_ip']
            protocol_type = df_test.iloc[index]['protocol_type']
            service = df_test.iloc[index]['service']
            flag = df_test.iloc[index]['flag']
            src_bytes = df_test.iloc[index]['src_bytes']
            dst_bytes = df_test.iloc[index]['dst_bytes']

            details = (f"{self.packet_counter}\t{src_ip}\t{dst_ip}\t{protocol_type}\t{service}\t{flag}\t"
                       f"{src_bytes}\t{dst_bytes}")
            summary = f"protocol_type: {protocol_type} | service: {service} | flag: {flag} | src_bytes: {src_bytes} | dst_bytes: {dst_bytes}"

            # pune în coadă pentru UI
            self.queue.put((ts, atype, conf, summary, details))
            # log la fișier
            with open(self.log_file, "a", encoding="utf-8") as f:
                f.write(f"{ts}\t{atype}\t{conf:.2f}%\n{details}\n{'-'*40}\n")
            time.sleep(3)  # Așteaptă 3 secunde înainte de a procesa următoarea mostră

    def handle_packet(self, pkt):
        if IP in pkt:
            elapsed_time = datetime.now() - self.start_time
            ts = f"{elapsed_time.seconds + elapsed_time.microseconds / 1e6:.4f}"
            try:
                atype, conf = preprocess_and_predict(pkt, threshold=self.threshold_var.get())
            except Exception as e:
                atype, conf = f"Eroare: {e}", 0.0

            additional_features = extract_additional_features(pkt)
            details = (f"{self.packet_counter}\t{additional_features['src_ip']}:{additional_features['src_port']}\t"
                       f"{additional_features['dst_ip']}:{additional_features['dst_port']}\t"
                       f"{pkt[IP].proto}\t{len(pkt)}\t{pkt.summary()}")
            # pune în coadă pentru UI
            self.queue.put((ts, atype, conf, pkt.summary(), details))
            # log la fișier
            with open(self.log_file, "a", encoding="utf-8") as f:
                f.write(f"{ts}\t{atype}\t{conf:.2f}%\n{details}\n{'-'*40}\n")
            # scrie pachetul în fișierul .pcapng
            wrpcap(self.pcap_file, [pkt], append=True)

if __name__ == "__main__":
    root = tk.Tk()
    app = LiveTrafficAnalyzer(root)
    root.mainloop()


# Cod pentru a testa, pe setul de antrenare, acuratețea și latența

In [31]:
import os
import tkinter as tk
from tkinter import scrolledtext
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Convolution1D, MaxPooling1D, LSTM, Dropout, Dense
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

MODEL_DIR = os.path.join(".", "saved_model")
MODEL_FILE_PATH = os.path.join(MODEL_DIR, "model_traffic_25.03.h5")

# Definirea coloanelor
columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent',
           'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
           'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
           'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
           'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
           'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
           'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack', 'level']

# Funcție de încărcare și preprocesare a datelor NSL-KDD
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path, delimiter=',', header=None)
    df.columns = columns  # Setează numele coloanelor

    X = df.iloc[:, :-1]  # Toate coloanele, mai puțin ultima
    y_raw = df['attack']  # Ultima coloană este eticheta

    # Convertim coloanele categorice
    label_encoders = {}
    for column in X.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column])
        label_encoders[column] = le

    # Convertim etichetele de ieșire
    label_encoder_y = LabelEncoder()
    y = label_encoder_y.fit_transform(y_raw)

    # Scala datele
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

    return X_scaled, y, label_encoder_y

# Crearea modelului CNN-LSTM
def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Convolution1D(128, 3, padding="same", activation="relu", input_shape=input_shape),
        Convolution1D(128, 3, padding="same", activation="relu"),
        Convolution1D(128, 3, padding="same", activation="relu"),
        MaxPooling1D(pool_size=2),
        LSTM(30),
        Dropout(0.1),
        Dense(num_classes, activation="softmax")
    ])
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
    return model

# Încărcare și pregătire date
X, y, label_encoder_y = load_and_preprocess_data("../Data/nsl-kdd/KDDTrain+.txt")
num_classes = len(np.unique(y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificăm dacă modelul este deja salvat
if os.path.exists(MODEL_FILE_PATH):
    model = load_model(MODEL_FILE_PATH, compile=False)
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
    model.summary()
    print("Modelul a fost încărcat din fișierul salvat.")
else:
    os.makedirs(MODEL_DIR, exist_ok=True)
    model = create_cnn_lstm_model(X_train.shape[1:], num_classes)
    history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))
    model.save(MODEL_FILE_PATH)
    model.summary()
    print("Modelul a fost antrenat și salvat în format .h5.")

    # Afișarea curbelor de antrenare
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Costul de antrenare')
    plt.plot(history.history['val_loss'], label='Costul de validare')
    plt.title('Curba de cost')
    plt.xlabel('Epoci')
    plt.ylabel('Cost')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Acuratețea de antrenare')
    plt.plot(history.history['val_accuracy'], label='Acuratețea de validare')
    plt.title('Curba de Acuratețe')
    plt.xlabel('Epoci')
    plt.ylabel('Acuratețe')
    plt.legend()

    plt.show()

# Evaluarea performanțelor modelului
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Performanțe model: Acuratețe = {accuracy * 100:.2f}%")

# Funcție de predicție detaliată
def preprocess_and_predict(data, threshold=50):
    prediction_probs = model.predict(data)[0]
    predicted_index = np.argmax(prediction_probs)
    confidence = prediction_probs[predicted_index] * 100

    # Clasificăm ca trafic legitim dacă încrederea este sub prag
    if confidence < threshold:
        return "Trafic Legitim", confidence

    # Obțineți categoria specifică de atac și denumirea etichetei
    predicted_category = label_encoder_y.inverse_transform([predicted_index])[0]
    return f"Atac: {predicted_category}", confidence

# Interfața Tkinter pentru analiza traficului
class LiveTrafficAnalyzer:
    def __init__(self, root):
        self.root = root
        self.root.title("Live Internet Traffic Analyzer")

        self.label = tk.Label(root, text="Internet Traffic Analysis", font=("Arial", 16))
        self.label.pack(pady=10)

        # Butoane de start și stop
        self.start_button = tk.Button(root, text="Start Analysis", command=self.start_analysis)
        self.start_button.pack(pady=5)
        self.stop_button = tk.Button(root, text="Stop Analysis", command=self.stop_analysis, state="disabled")
        self.stop_button.pack(pady=5)

        # ScrolledText pentru afișarea istoricului
        self.history_text = scrolledtext.ScrolledText(root, width=80, height=15, font=("Arial", 10))
        self.history_text.pack(pady=10)
        self.history_text.insert(tk.END, "Timestamp\t\tPredicted Attack Type\t\tConfidence (%)\n")
        self.history_text.insert(tk.END, "-"*60 + "\n")

        # Flag pentru controlul analizei continue
        self.running = False

    def start_analysis(self):
        self.running = True
        self.start_button.config(state="disabled")
        self.stop_button.config(state="normal")
        self.run_analysis()

    def stop_analysis(self):
        self.running = False
        self.start_button.config(state="normal")
        self.stop_button.config(state="disabled")

    def run_analysis(self):
        if self.running:
            sample_data = X_test[np.random.randint(0, len(X_test))].reshape(1, X_test.shape[1], 1)
            attack_type, confidence = preprocess_and_predict(sample_data)
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            self.history_text.insert(tk.END, f"{current_time}\t{attack_type}\t\t{confidence:.2f}%\n")
            self.history_text.see(tk.END)
            self.root.after(3000, self.run_analysis)

# Inițializarea și rularea aplicației Tkinter
root = tk.Tk()
app = LiveTrafficAnalyzer(root)
root.mainloop()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 42, 128)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 42, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 42, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 21, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 30)             │        19,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 23)             │           713 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,865 (464.32 KB)

 Trainable params: 118,865 (464.32 KB)

 Non-trainable params: 0 (0.00 B)

Modelul a fost încărcat din fișierul salvat.
788/788 ━━━━━━━━━━━━━━━━━━━━ 4:14 323ms/step - accuracy: 1.0000 - loss: 4.2705e- ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9972 - loss: 0.0071       ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9976 - loss: 0.00 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9980 - loss: 0.00 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9983 - loss: 0.00 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9985 - loss: 0.00 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9986 - loss: 0.00 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9987 - loss: 0.00 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9987 - loss: 0.00 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9988 - loss: 0.00 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9988 - loss: 0.00 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9988 - loss: 0.00 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9988 - loss: 0.00 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9988 - loss: 0.00 ━━━━━━━━━━━━━━━━━━━━ 

New

In [9]:
import os
import tkinter as tk
from tkinter import scrolledtext
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Dense, Flatten, Input, concatenate
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

MODEL_DIR = os.path.join(".", "saved_model")
MODEL_FILE_PATH = os.path.join(MODEL_DIR, "model_traffic_final_model_vgg_03.05.2025.h5")
LOG_DIR = os.path.join(".", "analysis_logs")

# Definirea coloanelor
columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent',
           'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
           'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
           'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
           'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
           'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
           'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack', 'level']

# Funcție de grupare a etichetelor de atac în cele 5 categorii
def group_attack_label(attack):
    attack = attack.lower()
    if attack in ['satan', 'portsweep', 'nmap', 'jpsweep']:
        return 'Probe'
    elif attack in ['spy', 'phf', 'multihop', 'imap', 'guess_passwd', 'ftp_write', 'warezmaster', 'warezclient']:
        return 'R2L'
    elif attack in ['rootkit', 'perl', 'loadmodule', 'buffer_overflow']:
        return 'U2R'
    elif attack in ['teardrop', 'smurf', 'pod', 'neptune', 'land', 'back']:
        return 'DoS'
    elif attack == 'normal':
        return 'NORMAL'
    else:
        return 'NORMAL'

# Funcție de încărcare și preprocesare a datelor de antrenare
def load_and_preprocess_train_data(file_path):
    df = pd.read_csv(file_path, delimiter=',', header=None)
    df.columns = columns

    X = df.iloc[:, :-1]
    y_raw = df['attack'].apply(group_attack_label)

    label_encoders = {}
    for column in X.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column])
        label_encoders[column] = le

    label_encoder_y = LabelEncoder()
    y = label_encoder_y.fit_transform(y_raw)

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)
    return X_scaled, y, label_encoders, label_encoder_y, scaler

# Funcție de încărcare și preprocesare a datelor de testare
# Se returnează și raw_test_df pentru afișarea pachetelor de date în interfață
def load_and_preprocess_test_data(file_path, label_encoders, label_encoder_y, scaler):
    df = pd.read_csv(file_path, delimiter=',', header=None)
    df.columns = columns
    # Stocăm datele originale înainte de filtrare
    raw_df = df.copy()

    y_raw = df['attack'].apply(group_attack_label)
    mask_attack = y_raw.isin(label_encoder_y.classes_)
    if not mask_attack.all():
        unknown_labels = y_raw[~mask_attack].unique()
        print("Următoarele etichete de atac nu sunt prezente în setul de antrenare și vor fi eliminate:", unknown_labels)
        df = df[mask_attack]
        raw_df = raw_df[mask_attack]  # Actualizăm și raw_df
        y_raw = y_raw[mask_attack]

    X = df.iloc[:, :-1]

    for column in X.select_dtypes(include=['object']).columns:
        le = label_encoders[column]
        mask_col = X[column].isin(le.classes_)
        if not mask_col.all():
            unknown_values = X.loc[~mask_col, column].unique()
            print(f"În coloana '{column}', următoarele valori necunoscute vor fi eliminate:", unknown_values)
            X = X[mask_col]
            y_raw = y_raw[mask_col]
            raw_df = raw_df[mask_col]
        X[column] = le.transform(X[column])

    y = label_encoder_y.transform(y_raw)

    X_scaled = scaler.transform(X)
    X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)
    return X_scaled, y, raw_df

# Definirea unui bloc Inception pentru date 1D
def inception_block(x, filters):
    branch1 = Conv1D(filters, kernel_size=1, padding='same', activation='relu')(x)

    branch3 = Conv1D(filters, kernel_size=1, padding='same', activation='relu')(x)
    branch3 = Conv1D(filters, kernel_size=3, padding='same', activation='relu')(branch3)

    branch5 = Conv1D(filters, kernel_size=1, padding='same', activation='relu')(x)
    branch5 = Conv1D(filters, kernel_size=5, padding='same', activation='relu')(branch5)

    branch_pool = MaxPooling1D(pool_size=3, strides=1, padding='same')(x)
    branch_pool = Conv1D(filters, kernel_size=1, padding='same', activation='relu')(branch_pool)

    x = concatenate([branch1, branch3, branch5, branch_pool], axis=-1)
    return x

# Crearea modelului VGG16IncepNet pentru date 1D
def create_vgg16incepnet_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = inception_block(x, filters=64)

    x = Conv1D(256, kernel_size=3, activation='relu', padding='same')(x)
    x = Conv1D(256, kernel_size=3, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.summary()  # Afișează sumarul modelului în consolă
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

# Încărcare și pregătire date
train_file = "../Data/nsl-kdd/KDDTrain+.txt"
test_file = "../Data/nsl-kdd/KDDTest+.txt"

X_train, y_train, label_encoders, label_encoder_y, scaler = load_and_preprocess_train_data(train_file)
X_test, y_test, raw_test_df = load_and_preprocess_test_data(test_file, label_encoders, label_encoder_y, scaler)

num_classes = len(np.unique(y_train))

if os.path.exists(MODEL_FILE_PATH):
    model = load_model(MODEL_FILE_PATH, compile=False)
    model.summary()
    print("Modelul a fost încărcat din fișierul salvat.")
else:
    os.makedirs(MODEL_DIR, exist_ok=True)
    model = create_vgg16incepnet_model(X_train.shape[1:], num_classes)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=100, batch_size=64,
              validation_data=(X_test, y_test), callbacks=[early_stopping])
    model.save(MODEL_FILE_PATH)
    model.summary()
    print("Modelul a fost antrenat și salvat în format .h5.")

def preprocess_and_predict(data, threshold=50):
    prediction_probs = model.predict(data)[0]
    predicted_index = np.argmax(prediction_probs)
    confidence = prediction_probs[predicted_index] * 100
    if confidence < threshold:
        return "Trafic Legitim", confidence
    predicted_category = label_encoder_y.inverse_transform([predicted_index])[0]
    return f"Atac: {predicted_category}", confidence

# Funcție de salvare a atacurilor detectate într-un fișier text
def save_attack_to_file(log_file, attack_type, confidence, packet_data):
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(log_file, 'a', encoding="utf-8") as file:
        file.write(f"{current_time}\t{attack_type}\t\t{confidence:.2f}%\n")
        file.write(f"Packet Data: {packet_data}\n")
        file.write("-"*60 + "\n")

# Interfața Tkinter pentru analiza traficului
class LiveTrafficAnalyzer:
    def __init__(self, root):
        self.root = root
        self.root.title("Live Internet Traffic Analyzer")

        self.label = tk.Label(root, text="Internet Traffic Analysis", font=("Arial", 16))
        self.label.pack(pady=10)

        # Butoane de start și stop
        self.start_button = tk.Button(root, text="Start Analysis", command=self.start_analysis)
        self.start_button.pack(pady=5)
        self.stop_button = tk.Button(root, text="Stop Analysis", command=self.stop_analysis, state="disabled")
        self.stop_button.pack(pady=5)

        # Zonă pentru istoricul clasificărilor
        self.history_text = scrolledtext.ScrolledText(root, width=80, height=10, font=("Arial", 10))
        self.history_text.pack(pady=10)
        self.history_text.insert(tk.END, "Timestamp\t\tPredicted Attack Type\t\tConfidence (%)\n")
        self.history_text.insert(tk.END, "-"*60 + "\n")

        # Zonă pentru afișarea detaliilor pachetului de date
        self.packet_text = scrolledtext.ScrolledText(root, width=80, height=10, font=("Arial", 10))
        self.packet_text.pack(pady=10)
        self.packet_text.insert(tk.END, "Detalii pachet:\n")
        self.packet_text.insert(tk.END, "-"*60 + "\n")

        self.running = False
        self.log_file = None  # Fișier pentru salvarea log-urilor

    def start_analysis(self):
        self.running = True
        self.start_button.config(state="disabled")
        self.stop_button.config(state="normal")
        # Creează un folder cu un nume bazat pe intervalul orar al începutului analizei
        os.makedirs(LOG_DIR, exist_ok=True)
        self.log_file = os.path.join(LOG_DIR, f"analysis_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
        self.run_analysis()

    def stop_analysis(self):
        self.running = False
        self.start_button.config(state="normal")
        self.stop_button.config(state="disabled")

    def run_analysis(self):
        if self.running:
            idx = np.random.randint(0, len(X_test))
            sample_data = X_test[idx].reshape(1, X_test.shape[1], 1)
            attack_type, confidence = preprocess_and_predict(sample_data)
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            # Actualizează istoricul în interfață
            self.history_text.insert(tk.END, f"{current_time}\t{attack_type}\t\t{confidence:.2f}%\n")
            self.history_text.see(tk.END)

            # Preluăm datele brute ale pachetului corespunzător indexului ales
            packet_details = raw_test_df.iloc[idx]
            # Construim un string cu pachetul de date pe un singur rând, separate prin " | "
            details_str = "Pachetul de date: " + " | ".join([f"{col}: {packet_details[col]}" for col in raw_test_df.columns])
            self.packet_text.delete("1.0", tk.END)
            self.packet_text.insert(tk.END, details_str + "\n" + "-"*60 + "\n")

            # Salvăm rezultatul analizei într-un fișier text
            save_attack_to_file(self.log_file, attack_type, confidence, details_str)

            self.root.after(3000, self.run_analysis)

# Inițializarea și rularea aplicației Tkinter
root = tk.Tk()
app = LiveTrafficAnalyzer(root)
root.mainloop()


În coloana 'attack', următoarele valori necunoscute vor fi eliminate: ['saint' 'mscan' 'apache2' 'snmpgetattack' 'processtable' 'httptunnel'
 'ps' 'snmpguess' 'mailbomb' 'named' 'sendmail' 'xterm' 'worm' 'xlock'
 'xsnoop' 'sqlattack' 'udpstorm']


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 42, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 42, 64)    │        256 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 42, 64)    │     12,352 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 21, 64)    │          0 │ conv1d_1[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 21, 128)   │     24,704 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 21, 128)   │     49,280 │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 10, 128)   │          0 │ conv1d_3[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 10, 64)    │      8,256 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 10, 64)    │      8,256 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_2     │ (None, 10, 128)   │          0 │ max_pooling1d_1[… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 10, 64)    │      8,256 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 10, 64)    │     12,352 │ conv1d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 10, 64)    │     20,544 │ conv1d_7[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_9 (Conv1D)   │ (None, 10, 64)    │      8,256 │ max_pooling1d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 10, 256)   │          0 │ conv1d_4[0][0],   │
│ (Concatenate)       │                   │            │ conv1d_6[0][0],   │
│                     │                   │            │ conv1d_8[0][0],   │
│                     │                   │            │ conv1d_9[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_10 (Conv1D)  │ (None, 10, 256)   │    196,864 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 10, 256)   │    196,864 │ conv1d_10[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_3     │ (None, 5, 256)    │          0 │ conv1d_11[0][0]   │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 1280)      │          0 │ max_pooling1d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │    655,872 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ dense[0][0]     

 Total params: 1,204,677 (4.60 MB)

 Trainable params: 1,204,677 (4.60 MB)

...

 Total params: 3,614,033 (13.79 MB)

 Trainable params: 1,204,677 (4.60 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,409,356 (9.19 MB)

Modelul a fost antrenat și salvat în format .h5.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [13]:
import os
import tkinter as tk
from tkinter import scrolledtext
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Dense, Flatten, Input, concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt

MODEL_DIR = os.path.join(".", "saved_model")
MODEL_FILE_PATH = os.path.join(MODEL_DIR, "model_traffic_final_model.h5")
LOG_DIR = os.path.join(".", "analysis_logs")

# Definirea coloanelor
columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent',
           'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
           'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
           'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
           'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
           'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
           'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack', 'level']

# Funcție de grupare a etichetelor de atac în cele 5 categorii
def group_attack_label(attack):
    attack = attack.lower()
    if attack in ['satan', 'portsweep', 'nmap', 'jpsweep']:
        return 'Probe'
    elif attack in ['spy', 'phf', 'multihop', 'imap', 'guess_passwd', 'ftp_write', 'warezmaster', 'warezclient']:
        return 'R2L'
    elif attack in ['rootkit', 'perl', 'loadmodule', 'buffer_overflow']:
        return 'U2R'
    elif attack in ['teardrop', 'smurf', 'pod', 'neptune', 'land', 'back']:
        return 'DoS'
    elif attack == 'normal':
        return 'NORMAL'
    else:
        return 'NORMAL'

# Funcție de încărcare și preprocesare a datelor de antrenare
def load_and_preprocess_train_data(file_path):
    df = pd.read_csv(file_path, delimiter=',', header=None)
    df.columns = columns

    X = df.iloc[:, :-1]
    y_raw = df['attack'].apply(group_attack_label)

    label_encoders = {}
    for column in X.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column])
        label_encoders[column] = le

    label_encoder_y = LabelEncoder()
    y = label_encoder_y.fit_transform(y_raw)

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)
    return X_scaled, y, label_encoders, label_encoder_y, scaler

# Funcție de încărcare și preprocesare a datelor de testare
# Se returnează și raw_test_df pentru afișarea pachetelor de date în interfață
def load_and_preprocess_test_data(file_path, label_encoders, label_encoder_y, scaler):
    df = pd.read_csv(file_path, delimiter=',', header=None)
    df.columns = columns
    # Stocăm datele originale înainte de filtrare
    raw_df = df.copy()

    y_raw = df['attack'].apply(group_attack_label)
    mask_attack = y_raw.isin(label_encoder_y.classes_)
    if not mask_attack.all():
        unknown_labels = y_raw[~mask_attack].unique()
        print("Următoarele etichete de atac nu sunt prezente în setul de antrenare și vor fi eliminate:", unknown_labels)
        df = df[mask_attack]
        raw_df = raw_df[mask_attack]  # Actualizăm și raw_df
        y_raw = y_raw[mask_attack]

    X = df.iloc[:, :-1]

    for column in X.select_dtypes(include=['object']).columns:
        le = label_encoders[column]
        mask_col = X[column].isin(le.classes_)
        if not mask_col.all():
            unknown_values = X.loc[~mask_col, column].unique()
            print(f"În coloana '{column}', următoarele valori necunoscute vor fi eliminate:", unknown_values)
            X = X[mask_col]
            y_raw = y_raw[mask_col]
            raw_df = raw_df[mask_col]
        X[column] = le.transform(X[column])

    y = label_encoder_y.transform(y_raw)

    X_scaled = scaler.transform(X)
    X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)
    return X_scaled, y, raw_df

# Definirea unui bloc Inception pentru date 1D
def inception_block(x, filters):
    branch1 = Conv1D(filters, kernel_size=1, padding='same', activation='relu')(x)

    branch3 = Conv1D(filters, kernel_size=1, padding='same', activation='relu')(x)
    branch3 = Conv1D(filters, kernel_size=3, padding='same', activation='relu')(branch3)

    branch5 = Conv1D(filters, kernel_size=1, padding='same', activation='relu')(x)
    branch5 = Conv1D(filters, kernel_size=5, padding='same', activation='relu')(branch5)

    branch_pool = MaxPooling1D(pool_size=3, strides=1, padding='same')(x)
    branch_pool = Conv1D(filters, kernel_size=1, padding='same', activation='relu')(branch_pool)

    x = concatenate([branch1, branch3, branch5, branch_pool], axis=-1)
    return x

# Crearea modelului VGG16IncepNet pentru date 1D
def create_vgg16incepnet_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)  # Adăugare Dropout pentru regularizare

    x = Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)  # Adăugare Dropout pentru regularizare

    x = inception_block(x, filters=64)
    x = Dropout(0.3)(x)  # Adăugare Dropout pentru regularizare

    x = Conv1D(256, kernel_size=3, activation='relu', padding='same')(x)
    x = Conv1D(256, kernel_size=3, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)  # Adăugare Dropout pentru regularizare

    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.summary()  # Afișează sumarul modelului în consolă

    # Ajustarea ratei de învățare
    optimizer = Adam(learning_rate=0.0001)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

# Încărcare și pregătire date
train_file = "../Data/nsl-kdd/KDDTrain+.txt"
test_file = "../Data/nsl-kdd/KDDTest+.txt"

X_train, y_train, label_encoders, label_encoder_y, scaler = load_and_preprocess_train_data(train_file)
X_test, y_test, raw_test_df = load_and_preprocess_test_data(test_file, label_encoders, label_encoder_y, scaler)

num_classes = len(np.unique(y_train))

if os.path.exists(MODEL_FILE_PATH):
    model = load_model(MODEL_FILE_PATH, compile=False)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate=0.0001), metrics=["accuracy"])
    model.summary()
    print("Modelul a fost încărcat din fișierul salvat.")
else:
    os.makedirs(MODEL_DIR, exist_ok=True)
    model = create_vgg16incepnet_model(X_train.shape[1:], num_classes)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=50, batch_size=32,
              validation_data=(X_test, y_test), callbacks=[early_stopping])
    model.save(MODEL_FILE_PATH)
    model.summary()
    print("Modelul a fost antrenat și salvat în format .h5.")

    # Afișarea curbelor de antrenare
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

# Evaluarea performanțelor modelului
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Performanțe model: Acuratețe = {accuracy * 100:.2f}%")

def preprocess_and_predict(data, threshold=50):
    prediction_probs = model.predict(data)[0]
    predicted_index = np.argmax(prediction_probs)
    confidence = prediction_probs[predicted_index] * 100
    if confidence < threshold:
        return "Trafic Legitim", confidence
    predicted_category = label_encoder_y.inverse_transform([predicted_index])[0]
    return f"Atac: {predicted_category}", confidence

# Funcție de salvare a atacurilor detectate într-un fișier text
def save_attack_to_file(log_file, attack_type, confidence, packet_data):
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(log_file, 'a', encoding="utf-8") as file:
        file.write(f"{current_time}\t{attack_type}\t\t{confidence:.2f}%\n")
        file.write(f"Packet Data: {packet_data}\n")
        file.write("-"*60 + "\n")

# Interfața Tkinter pentru analiza traficului
class LiveTrafficAnalyzer:
    def __init__(self, root):
        self.root = root
        self.root.title("Live Internet Traffic Analyzer")

        self.label = tk.Label(root, text="Internet Traffic Analysis", font=("Arial", 16))
        self.label.pack(pady=10)

        # Butoane de start și stop
        self.start_button = tk.Button(root, text="Start Analysis", command=self.start_analysis)
        self.start_button.pack(pady=5)
        self.stop_button = tk.Button(root, text="Stop Analysis", command=self.stop_analysis, state="disabled")
        self.stop_button.pack(pady=5)

        # Zonă pentru istoricul clasificărilor
        self.history_text = scrolledtext.ScrolledText(root, width=80, height=10, font=("Arial", 10))
        self.history_text.pack(pady=10)
        self.history_text.insert(tk.END, "Timestamp\t\tPredicted Attack Type\t\tConfidence (%)\n")
        self.history_text.insert(tk.END, "-"*60 + "\n")

        # Zonă pentru afișarea detaliilor pachetului de date
        self.packet_text = scrolledtext.ScrolledText(root, width=80, height=10, font=("Arial", 10))
        self.packet_text.pack(pady=10)
        self.packet_text.insert(tk.END, "Detalii pachet:\n")
        self.packet_text.insert(tk.END, "-"*60 + "\n")

        self.running = False
        self.log_file = None  # Fișier pentru salvarea log-urilor

    def start_analysis(self):
        self.running = True
        self.start_button.config(state="disabled")
        self.stop_button.config(state="normal")
        # Creează un folder cu un nume bazat pe intervalul orar al începutului analizei
        os.makedirs(LOG_DIR, exist_ok=True)
        self.log_file = os.path.join(LOG_DIR, f"analysis_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
        self.run_analysis()

    def stop_analysis(self):
        self.running = False
        self.start_button.config(state="normal")
        self.stop_button.config(state="disabled")

    def run_analysis(self):
        if self.running:
            idx = np.random.randint(0, len(X_test))
            sample_data = X_test[idx].reshape(1, X_test.shape[1], 1)
            attack_type, confidence = preprocess_and_predict(sample_data)
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            # Actualizează istoricul în interfață
            self.history_text.insert(tk.END, f"{current_time}\t{attack_type}\t\t{confidence:.2f}%\n")
            self.history_text.see(tk.END)

            # Preluăm datele brute ale pachetului corespunzător indexului ales
            packet_details = raw_test_df.iloc[idx]
            # Construim un string cu pachetul de date pe un singur rând, separate prin " | "
            details_str = "Pachetul de date: " + " | ".join([f"{col}: {packet_details[col]}" for col in raw_test_df.columns])
            self.packet_text.delete("1.0", tk.END)
            self.packet_text.insert(tk.END, details_str + "\n" + "-"*60 + "\n")

            # Salvăm rezultatul analizei într-un fișier text
            save_attack_to_file(self.log_file, attack_type, confidence, details_str)

            self.root.after(3000, self.run_analysis)

# Inițializarea și rularea aplicației Tkinter
root = tk.Tk()
app = LiveTrafficAnalyzer(root)
root.mainloop()


În coloana 'attack', următoarele valori necunoscute vor fi eliminate: ['saint' 'mscan' 'apache2' 'snmpgetattack' 'processtable' 'httptunnel'
 'ps' 'snmpguess' 'mailbomb' 'named' 'sendmail' 'xterm' 'worm' 'xlock'
 'xsnoop' 'sqlattack' 'udpstorm']


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 42, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_12 (Conv1D)  │ (None, 42, 64)    │        256 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_13 (Conv1D)  │ (None, 42, 64)    │     12,352 │ conv1d_12[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_4     │ (None, 21, 64)    │          0 │ conv1d_13[0][0]   │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 21, 64)    │          0 │ max_pooling1d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_14 (Conv1D)  │ (None, 21, 128)   │     24,704 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_15 (Conv1D)  │ (None, 21, 128)   │     49,280 │ conv1d_14[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_5     │ (None, 10, 128)   │          0 │ conv1d_15[0][0]   │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 10, 128)   │          0 │ max_pooling1d_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_17 (Conv1D)  │ (None, 10, 64)    │      8,256 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_19 (Conv1D)  │ (None, 10, 64)    │      8,256 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_6     │ (None, 10, 128)   │          0 │ dropout_6[0][0]   │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_16 (Conv1D)  │ (None, 10, 64)    │      8,256 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_18 (Conv1D)  │ (None, 10, 64)    │     12,352 │ conv1d_17[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_20 (Conv1D)  │ (None, 10, 64)    │     20,544 │ conv1d_19[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_21 (Conv1D)  │ (None, 10, 64)    │      8,256 │ max_pooling1d_6[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 10, 256)   │          0 │ conv1d_16[0][0],  │
│ (Concatenate)       │                   │            │ conv1d_18[0][0],  │
│                     │                   │            │ conv1d_20[0][0],  │
│                     │                   │            │ conv1d_21[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 10, 256)   │          0 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_22 (Conv1D)  │ (None, 10, 256)   │    196,864 │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_23 (Conv1D)  │ (None, 10, 256)   │    196,864 │ conv1d_22[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_7     │ (None, 5, 256)    │          0 │ conv1d_23[0][0]   │
│ (MaxPooling1D)      │                   │            │                 

 Total params: 1,204,677 (4.60 MB)

 Trainable params: 1,204,677 (4.60 MB)

 Non-trainable params: 0 (0.00 B)

Modelul a fost încărcat din fișierul salvat.
588/588 ━━━━━━━━━━━━━━━━━━━━ 3:25 350ms/step - accuracy: 0.9688 - loss: 0.34 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9566 - loss: 0.5077   ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9500 - loss: 0.52 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9490 - loss: 0.52 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9478 - loss: 0.54 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9469 - loss: 0.56 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9455 - loss: 0.57 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9444 - loss: 0.58 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9429 - loss: 0.60 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9414 - loss: 0.62 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9401 - loss: 0.63 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9390 - loss: 0.64 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9381 - loss: 0.65 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9373 - loss: 0.66 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/s